In [78]:
import time
import numpy

from matplotlib import pyplot
from tqdm.auto import tqdm
from ipywidgets import interact
from IPython.display import display, clear_output

from pysted import base, utils, raster, exp_data_gen

In [99]:

LASER_EX = {"lambda_" : 488e-9}
LASER_STED = {"lambda_" : 575e-9, "zero_residual" : 0.01}
DETECTOR = {"noise" : True}
OBJECTIVE = {}
# FLUO = {
#     "lambda_": 535e-9,
#     "qy": 0.6,
#     "sigma_abs": {488: 4.5e-21,   # was 1.15e-20
#                   575: 6e-21},
#     "sigma_ste": {560: 1.2e-20,
#                   575: 3.0e-22,   # was 6.0e-21
#                   580: 5.0e-21},
#     "sigma_tri": 1e-21,
#     "tau": 3e-09,
#     "tau_vib": 1.0e-12,
#     "tau_tri": 5e-6,
#     "phy_react": {488: 0.25e-7,   # 1e-4
#                   575: 25.0e-11},   # 1e-8
#     "k_isc": 0.26e6
# }
FLUO = {
    "lambda_": 535e-9,
    "qy": 0.6,
    "sigma_abs": {
        488: 0.2e-21,   # was 1.15e-20
        575: 0.01e-21
    },
    "sigma_ste": {
        575: 3.0e-22,   # was 6.0e-21
    },
    "sigma_tri": 0.01e-21, # 1e-21
    "tau": 3e-09,
    "tau_vib": 1.0e-12,
    "tau_tri": 5e-6,
    "phy_react": {488: 5.0e-5,   # 1e-4, 12e-7
                  575: 5.0e-8},   # 1e-8
    "k_isc": 0.26e+6
}

action_spaces = {
    "p_sted" : {"low" : 0., "high" : 350.0e-3},
    # "p_ex" : {"low" : 0., "high" : 100.0e-6},
    "p_ex" : {"low" : 0., "high" : 250.0e-6},
    "pdt" : {"low" : 10.0e-6, "high" : 150.0e-6},
}

percents_p_sted = [0.00, 0.10, 0.30, 0.50]
percents_p_ex = [0.05, 0.1, 0.25, 0.7, 0.9]

percent_pdt = 0.

def generate_bleach(sigma_tri, phy_488, phy_575, tau_vib=1.0, tau_tri=1.0, k_isc=0.26):
    numpy.random.seed(None)
    FLUO = {
        "lambda_": 535e-9,
        "qy": 0.6,
        "sigma_abs": {
            488: 0.2e-21,   # was 1.15e-20
            575: 0.01e-21
        },
        "sigma_ste": {
            575: 3.0e-22,   # was 6.0e-21
        },
        "sigma_tri": sigma_tri * 1e-21, # 1e-21
        "tau": 3e-09,
        "tau_vib": tau_vib * 1e-12,
        "tau_tri": tau_tri * 1e-6,
        "phy_react": {488: phy_488 * 1e-5,   # 1e-4, 12e-7
                      575: phy_575 * 1e-8},   # 1e-8
        "k_isc": k_isc * 1e+6
    }

    laser_ex = base.GaussianBeam(**LASER_EX)
    laser_sted = base.DonutBeam(**LASER_STED)
    detector = base.Detector(**DETECTOR)
    objective = base.Objective(**OBJECTIVE)
    fluo = base.Fluorescence(**FLUO)
    microscope = base.Microscope(laser_ex, laser_sted, detector, objective, fluo, load_cache=True)

    i_ex, i_sted, psf_det = microscope.cache(20e-9)

    percents_p_sted = [0]
    percents_p_ex = numpy.linspace(0.05, 0.95, num=9)

    acquisitions = []
    for percent_p_sted in percents_p_sted:
        for percent_p_ex in percents_p_ex:
            p_ex = percent_p_ex * (action_spaces["p_ex"]["high"] - action_spaces["p_ex"]["low"]) + action_spaces["p_ex"]["low"]
            p_sted = percent_p_sted * (action_spaces["p_sted"]["high"] - action_spaces["p_sted"]["low"]) + action_spaces["p_sted"]["low"]
            pdt = percent_pdt * (action_spaces["pdt"]["high"] - action_spaces["pdt"]["low"]) + action_spaces["pdt"]["low"]

            photons_ex = fluo.get_photons(i_ex * p_ex)
            k_ex = fluo.get_k_bleach(laser_ex.lambda_, photons_ex)
            duty_cycle = laser_sted.tau * laser_sted.rate
            photons_sted = fluo.get_photons(i_sted * p_sted * duty_cycle)
            k_sted = fluo.get_k_bleach(laser_sted.lambda_, photons_sted)

            prob_sted = numpy.ones(1)
            prob_ex = numpy.ones(1)

            prob_ex = prob_ex * numpy.prod(numpy.exp(-1. * k_ex * pdt))
            prob_sted = prob_sted * numpy.prod(numpy.exp(-1. * k_sted * pdt))
            prob = prob_ex * prob_sted

            initial_value = 125
            numpy.random.seed(42)
            current_value = numpy.random.binomial(initial_value, prob ** k_sted.size)

            acquisitions.append((initial_value - current_value) / initial_value)
    acquisitions = numpy.array(acquisitions)
    acquisitions = acquisitions.reshape(len(percents_p_sted), len(percents_p_ex))

    percents_p_sted = numpy.linspace(0.05, 0.95, num=9)
    percents_p_ex = [0.1]

    acquisitions_sted = []
    for percent_p_sted in percents_p_sted:
        for percent_p_ex in percents_p_ex:
            p_ex = percent_p_ex * (action_spaces["p_ex"]["high"] - action_spaces["p_ex"]["low"]) + action_spaces["p_ex"]["low"]
            p_sted = percent_p_sted * (action_spaces["p_sted"]["high"] - action_spaces["p_sted"]["low"]) + action_spaces["p_sted"]["low"]
            pdt = percent_pdt * (action_spaces["pdt"]["high"] - action_spaces["pdt"]["low"]) + action_spaces["pdt"]["low"]

            photons_ex = fluo.get_photons(i_ex * p_ex)
            k_ex = fluo.get_k_bleach(laser_ex.lambda_, photons_ex)
            duty_cycle = laser_sted.tau * laser_sted.rate
            photons_sted = fluo.get_photons(i_sted * p_sted * duty_cycle)
            k_sted = fluo.get_k_bleach(laser_sted.lambda_, photons_sted)
            
            fig, ax = pyplot.subplots()
            ax.imshow(k_sted)
            pyplot.show()

            prob_sted = numpy.ones(1)
            prob_ex = numpy.ones(1)

            prob_ex = prob_ex * numpy.prod(numpy.exp(-1. * k_ex * pdt))
            prob_sted = prob_sted * numpy.prod(numpy.exp(-1. * k_sted * pdt))
            prob = prob_ex * prob_sted

            initial_value = 125
            running_sum = 0
            current_value = numpy.random.binomial(initial_value, prob ** k_sted.size, size=10000)
                
            acquisitions_sted.append((initial_value - current_value.mean()) / initial_value)
    acquisitions_sted = numpy.array(acquisitions_sted)
    acquisitions_sted = acquisitions_sted.reshape(len(percents_p_sted), len(percents_p_ex))

    formatted = """
sigma_tri : {:0.4f}
phy_488 : {:0.4f}
phy_575 : {:0.4f}
tau_vib : {:0.4f}
tau_tri : {:0.4f}
k_isc : {:0.4f}
    """.format(sigma_tri, phy_488, phy_575, tau_vib, tau_tri, k_isc)
    print(formatted)
    print(acquisitions)
    print(acquisitions_sted.ravel())

#     return acquisitions

In [100]:
%matplotlib notebook
w = interact(
    generate_bleach, 
    sigma_tri=(0.1, 50., 0.01),
    phy_488=(0.0001, 0.01, 0.00001),
    phy_575=(0.00001, 0.01, 0.00001),
    tau_tri=(0.001, 5, 0.001),
    k_isc=(0.01, 5, 0.01)
)

# fig, ax = pyplot.subplots()
# ax.plot(percents_p_ex, acquisitions[0])
# ax.set(ylim=(-0.1, 1.1), xlim=(-0.1, 1.1))

interactive(children=(FloatSlider(value=25.05, description='sigma_tri', max=50.0, min=0.1, step=0.01), FloatSl…

In [73]:

LASER_EX = {"lambda_" : 488e-9}
LASER_STED = {"lambda_" : 575e-9, "zero_residual" : 0.01}
DETECTOR = {"noise" : True}
OBJECTIVE = {}
# FLUO = {
#     "lambda_": 535e-9,
#     "qy": 0.6,
#     "sigma_abs": {488: 4.5e-21,   # was 1.15e-20
#                   575: 6e-21},
#     "sigma_ste": {560: 1.2e-20,
#                   575: 3.0e-22,   # was 6.0e-21
#                   580: 5.0e-21},
#     "sigma_tri": 1e-21,
#     "tau": 3e-09,
#     "tau_vib": 1.0e-12,
#     "tau_tri": 5e-6,
#     "phy_react": {488: 0.25e-7,   # 1e-4
#                   575: 25.0e-11},   # 1e-8
#     "k_isc": 0.26e6
# }
FLUO = {
    "lambda_": 535e-9,
    "qy": 0.6,
    "sigma_abs": {
        488: 0.2e-21,   # was 1.15e-20
        575: 0.01e-21
    },
    "sigma_ste": {
        575: 3.0e-22,   # was 6.0e-21
    },
    "sigma_tri": 0.01e-21, # 1e-21
    "tau": 3e-09,
    "tau_vib": 1.0e-12,
    "tau_tri": 5e-6,
    "phy_react": {488: 5.0e-5,   # 1e-4, 12e-7
                  575: 5.0e-8},   # 1e-8
    "k_isc": 0.26e+6
}

action_spaces = {
    "p_sted" : {"low" : 0., "high" : 350.0e-3},
    # "p_ex" : {"low" : 0., "high" : 100.0e-6},
    "p_ex" : {"low" : 0., "high" : 250.0e-6},
    "pdt" : {"low" : 10.0e-6, "high" : 150.0e-6},
}

percents_p_sted = [0.00, 0.10, 0.30, 0.50]
percents_p_ex = [0.05, 0.1, 0.25, 0.7, 0.9]
percent_pdt = 0.

def generate_bleach(sigma_tri, phy_488, phy_575, tau_vib=1.0, tau_tri=1.0, k_isc=0.26):
    numpy.random.seed(None)
    FLUO = {
        "lambda_": 535e-9,
        "qy": 0.6,
        "sigma_abs": {
            488: 0.2e-21,   # was 1.15e-20
            575: 0.01e-21
        },
        "sigma_ste": {
            575: 3.0e-22,   # was 6.0e-21
        },
        "sigma_tri": sigma_tri * 1e-21, # 1e-21
        "tau": 3e-09,
        "tau_vib": tau_vib * 1e-12,
        "tau_tri": tau_tri * 1e-6,
        "phy_react": {488: phy_488 * 1e-5,   # 1e-4, 12e-7
                      575: phy_575 * 1e-8},   # 1e-8
        "k_isc": k_isc * 1e+6
    }

    laser_ex = base.GaussianBeam(**LASER_EX)
    laser_sted = base.DonutBeam(**LASER_STED)
    detector = base.Detector(**DETECTOR)
    objective = base.Objective(**OBJECTIVE)
    fluo = base.Fluorescence(**FLUO)
    microscope = base.Microscope(laser_ex, laser_sted, detector, objective, fluo, load_cache=True)

    i_ex, i_sted, psf_det = microscope.cache(20e-9)

    percents_p_ex = [0.05]

    acquisitions = []
    for percent_p_sted in percents_p_sted:
        for percent_p_ex in percents_p_ex:
            p_ex = percent_p_ex * (action_spaces["p_ex"]["high"] - action_spaces["p_ex"]["low"]) + action_spaces["p_ex"]["low"]
            p_sted = percent_p_sted * (action_spaces["p_sted"]["high"] - action_spaces["p_sted"]["low"]) + action_spaces["p_sted"]["low"]
            pdt = percent_pdt * (action_spaces["pdt"]["high"] - action_spaces["pdt"]["low"]) + action_spaces["pdt"]["low"]

            photons_ex = fluo.get_photons(i_ex * p_ex)
            k_ex = fluo.get_k_bleach(laser_ex.lambda_, photons_ex)
            duty_cycle = laser_sted.tau * laser_sted.rate
            photons_sted = fluo.get_photons(i_sted * p_sted * duty_cycle)
            k_sted = fluo.get_k_bleach(laser_sted.lambda_, photons_sted)

            prob_sted = numpy.ones(1)
            prob_ex = numpy.ones(1)

            prob_ex = prob_ex * numpy.prod(numpy.exp(-1. * k_ex * pdt))
            prob_sted = prob_sted * numpy.prod(numpy.exp(-1. * k_sted * pdt))
            prob = prob_ex * prob_sted

            initial_value = 125
            numpy.random.seed(42)
            current_value = numpy.random.binomial(initial_value, prob ** k_sted.size)

            acquisitions.append((initial_value - current_value) / initial_value)
    acquisitions = numpy.array(acquisitions)
    acquisitions = acquisitions.reshape(len(percents_p_sted), len(percents_p_ex))

#     clear_output()
#     print(acquisitions)
#     fig, ax = pyplot.subplots()
#     ax.plot(percents_p_ex, acquisitions[0])
#     ax.set(ylim=(-0.1, 1.1), xlim=(-0.1, 1.1))
#     pyplot.show()    

    formatted = """
sigma_tri : {:0.4f}
phy_488 : {:0.4f}
phy_575 : {:0.6f}
tau_vib : {:0.4f}
tau_tri : {:0.4f}
k_isc : {:0.4f}
    """.format(sigma_tri, phy_488, phy_575, tau_vib, tau_tri, k_isc)
    print(formatted)
    return acquisitions

In [77]:
%matplotlib notebook
w = interact(
    generate_bleach, 
    sigma_tri=30.32,
    phy_488=0.0007,
    phy_575=(0.00001, 0.001, 0.00001),
    tau_tri=3.08,
    k_isc=1.34
)

# fig, ax = pyplot.subplots()
# ax.plot(percents_p_ex, acquisitions[0])
# ax.set(ylim=(-0.1, 1.1), xlim=(-0.1, 1.1))

interactive(children=(FloatSlider(value=30.32, description='sigma_tri', max=90.96000000000001, min=-30.32), Fl…